<a href="https://colab.research.google.com/github/Eshan-K-Kaushal/Voice_Chatbot/blob/main/Colab/WOZ_Delivered_Concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# graph  - connect all tags
# make a list of the predicted tags in the previous transactions
# 0 - intro
# 1 - Reply to bot's question
# 2 - Asking the bot how's it doing
# 3 - Experience - in the new country
# 4 - Experience - in the old country
# 5 - Dad's job
# 6 - How do you support yourself
# 7 - Do you like your work
# 8 - Where are your right now - location
# 9 - What your future looks like
# 10- Talk of Kids
# 11- Favorite activity

# options
# colleagues, friends, justice, relationships, love, feelings, religion, traditions, culture.


import networkx
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from string import punctuation
!pip install emoji
import emoji
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import random
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

idk = [ 'I am sorry I dont understand, but this the best answer I could come up with.','I am sorry I dont understand.', 'Apologies but I dont understand your question', 'What is it again?']
gc = ['Yes!', 'Thanks!', 'I know right!']
# dict
dict_tags = {'A': {0:'intro'}, 'B':{1:'intro_reply'}, 'C':{2:'int_condition'}, 'D':{3:'ex_new_count'},
             'E': {4: 'ex_old_count'}, 'F':{5:'dads_job'}, 'G':{6:'support_urself'}, 'H':{7:'like_work'},
             'I':{8:'location'}, 'J':{9:'future_looks'}, 'K':{10:'kids'}, 'L':{11:'fave_activity'}, 'M':{12:'exit'}}

ls_succ = []

# graphs
graph = { "A" : ["A", "B", "C", "D", "E", "F","G","H","I","J","K","L",'M'],
          "B" : ["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "C" : ["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "D" : ["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "E" : ["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "F" : ["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "G":["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "H":["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "I":["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          "J":["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          'K':["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          'L':["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M'],
          'M':["A", "B", "C", "D", "E", "F","G","H","I","J","K","L", 'M']
        }

# getting the connected tags

#for node in graph:
    #print('for node: ', node)
    #for succ in graph[node]:
        #print(dict_tags[succ], list(dict_tags[succ].values())[0], sep=' <--------> ')
    #print('\n')

def generate_edges(graph):
    edges = []

    # for each node in graph
    for node in graph:

        # for each neighbour node of a single node
        print('for node: ', node)
        for neighbour in graph[node]:
            #print('For node: ',neighbour)
            # if edge exists then append
            edges.append((node, neighbour))
            #print('\n')
        #print(edges)
        edges = []
    #return edges

#print(generate_edges(graph))

#previous tags

prev_tags = []

data = pd.read_csv('/content/intents.csv', encoding = 'ISO-8859-1',
                   names=["sentiment","content"], skiprows=1)
# we don't want the first row
#print(data.head(10))
data_1 = data.copy()
data_1 = data_1[['sentiment', "content"]]
#print(data_1.head())

emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad',
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed',
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink',
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip',
          '=^.^=': 'cat', ':D': 'smile', ';D': 'smile'}

def cleaner(tweet):
    """Function to clean text data"""

    for emoj in emojis.keys():
        if emoj in tweet:
            tweet = tweet.replace(emoj, "emoji" + emojis[emoj])

    tweet = tweet.lower()
    tweet = ''.join(c for c in tweet if not c.isdigit())  # remove digits

    tweet = re.sub("@[A-Za-z0-9]+", "", tweet)  # Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)  # Remove http links

    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI)  # Remove Emojis

    tweet = tweet.replace("#", "").replace("_", " ")  # Remove hashtag sign but keep the text
    tweet = ''.join(c for c in tweet if c not in punctuation)  # remove all punctuation

    wordnet_lemmatizer = WordNetLemmatizer()  # with use of morphological analysis of words
    tweet = [wordnet_lemmatizer.lemmatize(word) for word in nltk.word_tokenize(tweet)]

    tweet = " ".join(w for w in tweet)
    return tweet

#data_1_small = data_1.head(90_000)
#data_2 = data_1_small.copy()

data_2 = data_1.copy()

# print(data_2[["sentiment","content"]])
# print(data_2.head())

data_2['content'] = data_2['content'].apply(cleaner)

# print(data_2[["sentiment","content"]])

X = data_2['content']
y = data_2['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=65)

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer()
tfidf = TfidfTransformer()
clf = LogisticRegression(max_iter=15000)

pipeline = Pipeline([
('vec', vectorizer),  # strings to token integer counts
('tfidf', tfidf),  # integer counts to weighted TF-IDF scores
('classifier', clf),  # train on TF-IDF vectors
])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print(classification_report(y_test, pred))
input_user = ''

while (input_user != 'terminate the code') or (input_user != 'terminate'):

    input_user_old = list(str(input('Type your sentence (USER): ')))
    input_user = "".join(input_user_old[0:len(input_user_old)])
    input_user2 = input_user
    tk_input_user = word_tokenize(input_user2)
    #print(tk_input_user)
    input_user = [input_user]
    #print(input_user)
    pred_test = pipeline.predict(input_user)
    #print(pred_test[0])
    prev_tags.append(pred_test[0])
    max_from_prev_tags = max(set(prev_tags), key = prev_tags.count)

    count = 0
    for i in range(len(prev_tags)):

        if prev_tags[i] == max_from_prev_tags:
            #print(prev_tags[i],'==',max_from_prev_tags)
            count += 1
        else:
            count = 0
            continue

    if count >= 3:
        count = 0
        print('We have talked about this a lot, lets move on. How about you ask me about something else?')

    if pred_test == [0]:
        print('Bot: Hi! How are you?')
    elif pred_test == [1]:
        print("That's great!")
    elif pred_test == [2]:
        print('I am doing good! Thanks for asking!')

    elif pred_test == [3]:
        print('It has been good so far. I am really happy here and pretty satisfied with my given condition! So many amenities here '
              'that really makes me feel that I have come to a better place and the future will be bright!')
        c = 0
        endin = 0
        while True:
            if endin >= 1:
                break
            if c >= 2:
                print('We have talked about this a lot! Lets talk about something else!')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            #input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)

            if ('do' in tk_input_user_new and 'think' in tk_input_user_new and 'parents' in tk_input_user_new) or (
                    'memories' in tk_input_user_new and 'of' in tk_input_user_new and 'parents' in tk_input_user_new) or (
                    'stay' in tk_input_user_new and 'in' in tk_input_user_new and 'touch' in tk_input_user_new and 'with' in tk_input_user_new and 'parents' in tk_input_user_new) or (
                    'do' in tk_input_user_new and 'miss' in tk_input_user_new and 'parents' in tk_input_user_new):
                c += 1
                print('Yes I sure do! Initially, I did use to miss them a lot! '
                      'But one slowly gets used to it. I try my best to stay in touch with them by calling them every evening.')

            if ('think' in tk_input_user_new and 'of' in tk_input_user_new and 'future' in tk_input_user_new) or (
                    'think' in tk_input_user_new and 'about' in tk_input_user_new and 'future' in tk_input_user_new
            ) or ('positive' in tk_input_user_new and 'about' in tk_input_user_new and 'future' in tk_input_user_new) or (
                'of' in tk_input_user_new and 'future' in tk_input_user_new) or ('about' in tk_input_user_new and 'future' in tk_input_user_new):
                c += 1
                print('Yes I am really positive about my future! I believe I will be able to thrive and live a successful life here sans any fear of violence or crimes')

            if ('tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new):
                c += 1
                print('The US is proving to be a really comfortable place for me! People are really nice and helpful and I really thought the life here is going '
                      'to be really busy but is more than manageable! The place is full of opportunities and I am really positive about my future here!')

            else:
                print('What was it again?')
                endin += 1
                break

    elif pred_test == [4]:
        print('It was not that good back there. A sense of fear always accompanied me anywhere I went. The government was corrupt. '
              'It was not good for people who belonged to the lower classes of the society or were poor. Since my dad was an offcial in the '
              'foreign administrative services, I never felt how wrong the things were. But my dad always used to point out the faults '
              'in our governments systems that was causing the poor to go poorer and the rich to get richer. The crimes were present everywhere and were increasing '
              'at an alarming rate.')
        c = 0
        endin = 0
        while True:
            if endin >= 1:
                break
            if c >= 1:
                print('We have talked about this a lot! Lets talk about something else!')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            #input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)
            if('tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new) or ('want' in tk_input_user_new and 'know' in tk_input_user_new and 'more' in tk_input_user_new and 'this' in tk_input_user_new):
                c += 1
                print('Due the government being corrupt and the crimes increasing at a rapid rate, the country was not safe at all and that is why my mom and dad decided '
                      'to send me out to the States so that I could pursue my dreams with no fear.')
            else:
                print('What was it again?')
                endin += 1
                continue

    elif pred_test == [5]:
        print('My Dad worked as an Official in the Foreign Administrative Services. His name is Ezikiel and he is currently 52 years old.')
        c = 0
        endin = 0
        while True:
            if endin >= 1:
                break
            if c >= 2:
                print('We have talked about this a lot! Lets talk about something else!')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            #input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)
            if ('old' in tk_input_user_new and 'is' in tk_input_user_new and 'he' in tk_input_user_new) or ('his' in tk_input_user_new and 'age' in tk_input_user_new):
                c += 1
                print('He is 52 years old')

            if ('is' in tk_input_user_new and 'his' in tk_input_user_new and 'name' in tk_input_user_new) or ('is' in tk_input_user_new and 'their' in tk_input_user_new and 'name' in tk_input_user_new):
                c += 1
                print('His name is Ezikiel')

            if ('care' in tk_input_user_new and 'of' in tk_input_user_new and 'you' in tk_input_user_new):
                c += 1
                print('Yes! Ofcourse he did!')

            if ('tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new):
                c += 1
                print('My dad is a really smart person - really resourceful, and a really caring man! He loves my mom and his family like anything!')

            if ('tell' in tk_input_user_new and 'about' in tk_input_user_new and 'mom' in tk_input_user_new) or (
                    'tell' in tk_input_user_new and 'about' in tk_input_user_new and 'mother' in tk_input_user_new) or (
                    'talk' in tk_input_user_new and 'about' in tk_input_user_new and 'mom' in tk_input_user_new) or (
                    'about' in tk_input_user_new and 'mom' in tk_input_user_new) or ('about' in tk_input_user_new and 'mother' in tk_input_user_new): \
                    print(
                        'Her name is Emmaline and I love her! She is the best person i have known so far and she is the reason what I am what I am right now! '
                        'She really took care of me and did a great job by pushing me through all the tough times and the hard times! I do miss her and my dad too!')
            if ('you' in tk_input_user_new and 'love' in tk_input_user_new and 'her' in tk_input_user_new) \
                or ('you' in tk_input_user_new and 'love' in tk_input_user_new and 'him' in tk_input_user_new) \
                    or ('you' in tk_input_user_new and 'love' in tk_input_user_new and 'them' in tk_input_user_new):
                print('Yes! I love them more than anything!')
            else:
                print(random.choice(idk))
                endin += 1
                continue

    elif pred_test == [6]:
        print('I work as a junior mechanical engineer at Chevrolet right now. It is a really good company and I like the workplace environment there.'
              ' Much better than what we use to have it back there in the old country.')

        c = 0
        endin = 0
        while True:
            if endin >= 1:
                break
            if c >= 2:
                print('We have talked about this a lot! Lets talk about something else!')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            #input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)
            if ('tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new) or ('please' in tk_input_user_new and 'explain' in tk_input_user_new) or (
                    'know' in tk_input_user_new and 'more' in tk_input_user_new) or ('more' in tk_input_user_new and 'about' in tk_input_user_new and 'it' in tk_input_user_new) or (
                'explain' in tk_input_user_new and 'more' in tk_input_user_new):
                c += 1
                print('Right now as a junior mechnical engineer, my job is assure quality of the parts of the engine we make, my specific duty is to check the design and quality of the '
                      'pistons of the engines, their cylinders and the carburetors. My duty as a senior menchanical engineer will allow me to work with the R&D team - which'
                      ' has been a dream of mine since I was a kid!')
            else:
                print(random.choice(idk))
                break

    elif pred_test == [7]:
        print('I like it! I would like to work as a senior mechanical engineer to be honest! I mean I also did my bachelors in mechanical engineering. I love cars and machines.'
              'So, I am pursuing my masters in the field of mechanical engineering too just so I can be eligible for the post of senior mechanical engineer.')
        c = 0
        endin = 0
        while True:
            if endin >= 1:
                break
            if c >= 2:
                print('We have talked about this a lot! Lets talk about something else!')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            #input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)
            if ('what' in tk_input_user_new and 'college' in tk_input_user_new and 'you' in tk_input_user_new) or ('which' in tk_input_user_new and 'university' in tk_input_user_new) or (
                    'which' in tk_input_user_new and 'college' in tk_input_user_new) or ('where' in tk_input_user_new and 'doing' in tk_input_user_new and 'from' in tk_input_user_new):
                c += 1
                print('I am pursuing my masters from Vanderbuilt University in Nashville, Tennessee.')

            if ('how' in tk_input_user_new and 'is' in tk_input_user_new and 'experience' in tk_input_user_new) or ('tell' in tk_input_user_new and 'about' in tk_input_user_new and 'experience' in tk_input_user_new) or ('how' in tk_input_user_new and 'you' in tk_input_user_new and 'like' in tk_input_user_new):
                c += 1
                print('I love it so far! I mean really good faculty, made tonnes of new friends and the college has some really good extra-curricular activities.')

            if ('what' in tk_input_user_new and 'is' in tk_input_user_new and 'your' in tk_input_user_new and 'job' in tk_input_user_new) or \
                    ('tell' in tk_input_user_new and 'about' in tk_input_user_new and 'duties' in tk_input_user_new) or \
                    ('explain' in tk_input_user_new and 'your' in tk_input_user_new and 'job' in tk_input_user_new) or (
                    'tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new) or (
                'what' in tk_input_user_new and 'do' in tk_input_user_new and 'your' in tk_input_user_new and 'job' in tk_input_user_new):
                c += 1

                print('Right now as a junior mechnical engineer, my job is assure quality of the parts of the engine we make, my specific duty is to check the design and quality of the '
                      'pistons of the engines, their cylinders and the carburetors. My duty as a senior menchanical engineer will allow me to work with the R&D team - which'
                      ' has been a dream of mine since I was a kid!')
            else:
                print(random.choice(idk))
                endin += 1
                continue

    elif pred_test == [8]:
        print('I am currently talking to you in this here museum')

    elif pred_test == [9]:
        print('I am pretty positive about it! Once I am done with my masters, I will find a nice job for myself and then its all never back down from there!')
        c = 0
        endin = 0
        while True:
            if endin >= 2:
                break
            if c>= 2:
                print('We have talked about this a lot! Lets talk about something else!')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            #input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)
            if ('what' in tk_input_user_new and 'company' in tk_input_user_new and 'you' in tk_input_user_new) or \
                    ('which' in tk_input_user_new and 'comapny' in tk_input_user_new) or \
                    ('any' in tk_input_user_new and 'companies' in tk_input_user_new):
                c += 1
                print('I would like to work for Dodge, Jeep, Chrysler, any leading car manufacturing company, to be honest. Even Sabelt would work!')
            if ('you' in tk_input_user_new and 'willing' in tk_input_user_new and 'relocate' in tk_input_user_new) or ('would' in tk_input_user_new and 'you' in tk_input_user_new and 'relocate' in tk_input_user_new) or ('plans' in tk_input_user_new and 'to' in tk_input_user_new and 'relocate' in tk_input_user_new):
                c += 1
                print('Given the fact that most car manufacturing companies are in Detroit, I would not mind relocating, plus it is always more exposure I am after!')
            if ('tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new):
                print('Thats all, I mean whats more to say!')
            else:
                print(random.choice(idk))
                endin += 1
                continue

    elif pred_test == [10]:
        print('I have 4 kids: 2 sons and 2 daughters. Their names are Alfred, Bart, Casey, and Martina. I have a lovely wife - Freya and we have been married for 8 years now!')
        c = 0
        endin = 0
        while True:
            if endin >= 1:
                break
            if c >= 2:
                print('We have talked about this a lot! Lets talk about something else.')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            # input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)
            if ('what' in tk_input_user_new and 'schools' in tk_input_user_new and 'go' in tk_input_user_new) or (
                    'schools' in tk_input_user_new and 'go' in tk_input_user_new) or (
                    'any' in tk_input_user_new and 'schools' in tk_input_user_new):
                c += 1
                print('They all go to the same school - Nashville Elementary School')
            if ('I' in tk_input_user_new and 'am' in tk_input_user_new and 'happy' in tk_input_user_new) or (
                    'so' in tk_input_user_new and 'happy' in tk_input_user_new and 'you' in tk_input_user_new) or (
                    'thats' in tk_input_user_new and 'great' in tk_input_user_new):
                c += 1
                print('Thanks!')
            if ('tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new) or ('know' in tk_input_user_new and 'more' in tk_input_user_new) or(
                    'talk' in tk_input_user_new and 'more' in tk_input_user_new) or (
                    'about' in tk_input_user_new and 'mom' in tk_input_user_new and 'dad' in tk_input_user_new):
                c += 1
                print("My dad's name is Ezikiel and my mom's name is Emmaline. I have 1 brother as a sibling "
                      "and he is 4 years younger to me. He is currently pursuing his bachelors in the field of chemical engineering from Alberta University in Canada. "
                      "We practice the religion of Christianity and are Catholics. My dad works as an official in the Foreign Adminitrative Services and my mother is a "
                      "house wife. I love my parents and my brother since they have supported me a lot through all the tough and the hard times!")
            if ('tell' in tk_input_user_new and 'about' in tk_input_user_new and 'mom' in tk_input_user_new) or (
                    'tell' in tk_input_user_new and 'about' in tk_input_user_new and 'mother' in tk_input_user_new) or (
                    'talk' in tk_input_user_new and 'about' in tk_input_user_new and 'mom' in tk_input_user_new):\
                print('Her name is Emmaline and I love her! She is the best person i have known so far and she is the reason what I am what I am right now! '
                      'She really took care of me and did a great job by pushing me through all the tough times and the hard times! I do miss her and my dad too!')
            else:
                print(random.choice(idk))
                endin += 1
                continue

    elif pred_test == [11]:
        print('I love to race cars on the weekends. Being a mechanical engineer for Ford allows me to test'
              'cars on a regular basis')
        input_user_in = list(str(input('Type your sentence (USER): ')))
        input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
        #input_user_in_new = [input_user_in_new]
        tk_input_user_new = word_tokenize(input_user_in_new)
        if 'long' in tk_input_user_new and 'doing' in tk_input_user_new and 'it' in tk_input_user_new:
            print('I have been doing it for the past 5 years now!')
        else:
            print('Alright! So...What was it again?')

    elif pred_test == [12]:
        print('Bye now!')
        break

    elif pred_test == [13]:
        print('Yes I sure do! Initially, I did use to miss them a lot! '
              'But one slowly gets used to it. I try my best to stay in touch with them by calling them every evening.')
        c = 0
        endin = 0
        while True:

            if endin >= 1:
                break
            if c >= 2:
                print('We have talked about this a lot! Lets talk about something else.')
                break
            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            # input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)
            if ('what' in tk_input_user_new and 'use' in tk_input_user_new and 'talk' in tk_input_user_new and 'parents' in tk_input_user_new) or (
                    'apps' in tk_input_user_new and 'do' in tk_input_user_new and 'you' in tk_input_user_new and 'use' in tk_input_user_new and 'talk' in tk_input_user_new and 'parents' in tk_input_user_new) or (
                    'what' in tk_input_user_new and 'apps' in tk_input_user_new) or ('how' in tk_input_user_new and 'stay' in tk_input_user_new and 'touch' in tk_input_user_new):
                c += 1
                print('Normally I FaceTime them, or WhatsApp call them')
            if ('how' in tk_input_user_new and 'frequently' in tk_input_user_new and 'talk' in tk_input_user_new) or (
                    'how' in tk_input_user_new and 'often' in tk_input_user_new and 'you' in tk_input_user_new and 'speak' in tk_input_user_new) or (
                    'how' in tk_input_user_new and 'frequently' in tk_input_user_new) or (
                    'how' in tk_input_user_new and 'often' in tk_input_user_new):
                c += 1
                print('As I have said before, almost everyday!')
            else:
                print(random.choice(idk))
                endin += 1
                continue

    elif pred_test == [14]:
        print('I love Italian! Anything Italian - especially pizzas and pasta! I love Chicken Alfredo from Olive Garden and nothing other than that to be honest!')
        c = 0
        endin = 0
        while True:

            if c >= 2:
                print('We have talked about this a lot! Lets talk about something else.')
                break

            if endin >= 1:
                break

            input_user_in = list(str(input('Type your sentence (USER): ')))
            input_user_in_new = "".join(input_user_in[0:len(input_user_in)])
            # input_user_in_new = [input_user_in_new]
            tk_input_user_new = word_tokenize(input_user_in_new)

            if (
                    'any' in tk_input_user_new and 'other' in tk_input_user_new and 'food' and 'like' in tk_input_user_new) or (
                    'any' in tk_input_user_new and 'other' in tk_input_user_new and 'dish' in tk_input_user_new and 'you' in tk_input_user_new and 'like' in tk_input_user_new and 'parents' in tk_input_user_new) or (
                    'other' in tk_input_user_new and 'thing' in tk_input_user_new and 'like' in tk_input_user_new) or (
                'any' in tk_input_user_new and 'thing' in tk_input_user_new and 'else' in tk_input_user_new) or ('what' in tk_input_user_new and 'else' in tk_input_user_new) or (
                    'anything' in tk_input_user_new and 'else' in tk_input_user_new) or ('anything' in tk_input_user_new and 'other' in tk_input_user_new)\
                    or ('any' in  tk_input_user_new and 'thing' in tk_input_user_new and 'else' in tk_input_user_new) or ('any' in  tk_input_user_new and 'thing' in tk_input_user_new and 'other' in tk_input_user_new):
                c += 1
                print('Nah... I like the Chicken Alfredo a bit too much!')

            if (    'tell' in tk_input_user_new and 'traditional' in tk_input_user_new and 'about' and 'food' in tk_input_user_new) or (
                    'talk' in tk_input_user_new and 'about' in tk_input_user_new and 'traditional' in tk_input_user_new and 'food' in tk_input_user_new ) or (
                    'food' in tk_input_user_new and 'eat' in tk_input_user_new and 'country' in tk_input_user_new) or (
                    'food' in tk_input_user_new and 'eat' in tk_input_user_new and 'where' in tk_input_user_new and 'you' in tk_input_user_new and 'from' in tk_input_user_new) \
                    or ('food' in tk_input_user_new and 'eat' in tk_input_user_new and 'back' in tk_input_user_new and 'there' in tk_input_user_new) or (
                'tell' in tk_input_user_new and 'more' in tk_input_user_new) or ('tell' in tk_input_user_new and 'me' in tk_input_user_new and 'more' in tk_input_user_new):
                c += 1
                print('Back in my country we have amazing dishes! We have Stuffed Cabbage Rolls, Salo, Banush, Potato Pancakes and yes, Vodka!')

            if ('sounds' in tk_input_user_new and 'good' in tk_input_user_new) or ('wow' in tk_input_user_new):
                print('I know right! I like that food too! But not as much as my chicken alfredo')

            else:
                print(random.choice(idk))
                endin += 1
                continue

    elif pred_test == [15]:

        print('My countrys strong tradition of folk art and embroidery continues to this day, with Ukrainian embroidery often '
              'considered an art form in itself. Ukrainian customs are heavily influenced by the Ukrainian Greek Catholicism, Ruthenian Greek Catholicism '
              'and Eastern Orthodox Church and traditions from Slavic mythology. Oh God! I love my culture!')

    elif pred_test == [16]:
        print('Good!')

    elif pred_test == [17]:
        print(random.choice(gc))

    elif pred_test == [18]:
        print('Alright!')

    else:
        print('I am sorry I dont have any answers to that as of now. Maybe the future updates will allow me to answer '
              'such questions!')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         2
           3       0.60      1.00      0.75         3
           4       1.00      0.67      0.80         6
           5       1.00      0.75      0.86         4
           6       0.33      1.00      0.50         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         3
          11       0.67      1.00      0.80         2
          12       1.00      1.00      1.00         2
          13       1.00      0.75      0.86         4
          14       1.00      0.75      0.86         4
          15       1.00      1.00      1.00         1
          17       1.00      1.00      1.00         2
          18       1.00      1.00      1.00         1

    accuracy              

KeyboardInterrupt: ignored